# Cálculo de intervalo de confiança de acordo com o artigo "On the accuracy of simulated percentage points" (Juritz et al).

Nomenclatura usada no artigo:

- $f(x)$: pdf de uma variável aleatória $X$;
- $0 < p < 1$, ${\xi}_p$ denota o (100$p$) percentil de $X$;


## Estimativa do percentil e intervalo de confiança

Toda a discussão nessa seção é um resumo/tradução dos dados do artigo.

### The single-sample method

Nesse modelo o percentil é estimado a partir de uma única simulação de tamanho $n$.

Seja $X_{(i)}$ a $i$'ésima menor observação da simulação. De acordo com o artigo, David (1970) mostrou que para qualquer variável aleatória contínua o irtevalo $(X_{(r)}, X_{(s)})$, com $r < s$, abarca ${\xi}_p$ com probabilidade $ \pi(r, s, n, p) $. Essa probabilidade é dada por:


<center>$Prob(X_{(r)} \lt \xi_p \lt X_{(s)}) = \pi(r, s, n, p) = \sum_{i = 1}^{s - 1} \binom{n}{p} p^i (1 - p)^{(n-i)}$</center>


Note que o que está sendo feito aqui é:

1. fazemos uma simulação de Monte Carlo e o resultado são amostras da variável aleatória $X$; São realizadas $n$ simulações.

2. ordenamos o resultado do maior para o menor e chamamos de $X_{(i)}$ onde $ 1 <= i <= n $ é o índice dos resultados já ordenados;

3. estimamos o valor do (100$p$) percentil de $X$: ${\xi}_p \sim X_{(k)}$, onde $k = ceil(np + 1)$.

4. calculamos dois índices, $r$ e $s$. Esses índices, quando aplicados em $X_{(i)}$, dão o intervalo de confiança para a nossa estimativa do percentil com probabilidade $\pi(r,s,n,p)$

$p$ é o percentil (ou quantil) que estamos interessados. Assim, $p = Prob(X \le \xi_p)$. $r$ e $s$ são **escolhidos** de tal forma que o intervalo tenha confiança de $ 1 - \alpha$, ou seja, $\pi(r,s,n,p) = 1 - \alpha$.

Considerando $n$ como um valor muito grande, $\pi(r,s,n,p)$ pode ser calculado aproximando a distribuação binomial por uma normal. Com isso, $r$ e $s$ pode ser encontrado como:

<center>
    $r = -w z_\gamma + np + \frac{1}{2}$ e $s = +w z_\gamma + np + \frac{1}{2}$
</center>

onde $w = \sqrt{np(1-p)}$ e $z_\gamma$ se refere ao (100$\gamma$) percentil da distribuição normal padrão. Por fim, $\gamma = 1 - \

<span style="color:red">OBSERVAÇÃO: Essa aproximação é válida quando $np > 5$ e $nq > 5$ (onde $q = 1 - p$) [Referência](https://stats.libretexts.org/Courses/Las_Positas_College/Math_40%3A_Statistics_and_Probability/06%3A_Continuous_Random_Variables_and_the_Normal_Distribution/6.04%3A_Normal_Approximation_to_the_Binomial_Distribution). Como exemplo, se estamos trabalhando com o quantil 999, $ p = 0.999$ e $q = 0.001$, então $n$ deve ser no mínimo 5000.</span>